In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Melanoma"
cohort = "GSE148319"

# Input paths
in_trait_dir = "../../input/GEO/Melanoma"
in_cohort_dir = "../../input/GEO/Melanoma/GSE148319"

# Output paths
out_data_file = "../../output/preprocess/Melanoma/GSE148319.csv"
out_gene_data_file = "../../output/preprocess/Melanoma/gene_data/GSE148319.csv"
out_clinical_data_file = "../../output/preprocess/Melanoma/clinical_data/GSE148319.csv"
json_path = "../../output/preprocess/Melanoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE148319_family.soft.gz', 'GSE148319_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Melanoma/GSE148319/GSE148319_family.soft.gz
Matrix file: ../../input/GEO/Melanoma/GSE148319/GSE148319_series_matrix.txt.gz


Background Information:
!Series_title	"Dietary palmitic acid promotes a prometastatic epigenetic memory related to tumor innervation [IV]"
!Series_summary	"Metastasis is promoted by fatty acid (FA) uptake and metabolism1-2. How this works, or whether all dietary FAs are prometastatic, is not known. Here we show that dietary palmitic acid (PA), but not oleic acid (OA) or linoleic acid, promotes metastasis, indicating specificity of action for distinct FAs. Strikingly, tumours acutely exposed to a PA–rich diet remain highly metastatic even when serially transplanted. This PA–induced prometastatic memory requires the FA transporter CD36 as well as the epigenetically stable deposition of histone H3 lysine 4 trimethylation by the methyltransferase Set1A/COMPASS. Genes with this metastatic memory predominantly relate to a neural signature that stimulates intratumor oligodendrogenesis and perineural invasion, two parameters strongly correlated with metastasis but etiologically poorly understo

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data from oral carcinoma and melanoma cell lines
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify rows for trait, age, and gender
# Looking at sample characteristics dictionary, I can identify melanoma samples in row 3 (biological replicate)
trait_row = 3
# Age and gender information are not available in this dataset
age_row = None
gender_row = None

# 2.2 Define conversion functions
def convert_trait(value):
    """
    Convert trait data to binary format (1 for Melanoma, 0 for non-Melanoma)
    """
    if value is None:
        return None
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if the value contains "Melanoma"
    if "Melanoma" in value:
        return 1
    else:
        return 0

def convert_age(value):
    """
    Convert age data to continuous format
    Not used in this dataset as age data is not available
    """
    if value is None:
        return None
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """
    Convert gender data to binary format (0 for female, 1 for male)
    Not used in this dataset as gender data is not available
    """
    if value is None:
        return None
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value in ["female", "f"]:
        return 0
    elif value in ["male", "m"]:
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Check if clinical data is available (trait_row is not None)
if trait_row is not None:
    # Use the sample characteristics dictionary to create a clinical dataframe
    # First, get the sample characteristics from the matrix file
    matrix_file = f"{in_cohort_dir}/GSE148319_series_matrix.txt.gz"
    
    # We'll extract the sample characteristics data from the matrix file
    sample_chars = {}
    with gzip.open(matrix_file, 'rt') as f:
        for line in f:
            if line.startswith('!Sample_characteristics_ch1'):
                parts = line.strip().split('\t')
                for i, part in enumerate(parts[1:], 1):
                    if i not in sample_chars:
                        sample_chars[i] = []
                    sample_chars[i].append(part.strip('"'))
            # Stop reading once we reach the data section
            if line.startswith('!series_matrix_table_begin'):
                break
    
    # Convert to dataframe
    clinical_data = pd.DataFrame(sample_chars)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{1: [0.0], 2: [0.0], 3: [0.0], 4: [0.0], 5: [0.0], 6: [0.0], 7: [0.0], 8: [0.0], 9: [0.0], 10: [0.0], 11: [0.0], 12: [0.0], 13: [0.0], 14: [0.0], 15: [0.0], 16: [0.0], 17: [0.0], 18: [0.0], 19: [0.0], 20: [0.0], 21: [0.0], 22: [0.0], 23: [0.0], 24: [0.0], 25: [0.0], 26: [0.0], 27: [0.0], 28: [0.0], 29: [0.0], 30: [0.0], 31: [0.0], 32: [0.0], 33: [0.0], 34: [0.0], 35: [1.0], 36: [1.0], 37: [1.0], 38: [1.0], 39: [1.0], 40: [1.0], 41: [1.0], 42: [1.0], 43: [1.0], 44: [1.0], 45: [1.0], 46: [1.0], 47: [1.0], 48: [1.0], 49: [1.0], 50: [1.0], 51: [0.0], 52: [0.0], 53: [0.0], 54: [0.0], 55: [0.0], 56: [0.0], 57: [0.0], 58: [0.0], 59: [0.0], 60: [0.0], 61: [0.0], 62: [0.0], 63: [0.0], 64: [0.0], 65: [0.0], 66: [0.0], 67: [0.0], 68: [0.0], 69: [0.0], 70: [0.0], 71: [0.0], 72: [0.0], 73: [0.0], 74: [0.0], 75: [0.0], 76: [0.0], 77: [0.0], 78: [0.0], 79: [0.0], 80: [0.0], 81: [0.0], 82: [0.0], 83: [0.0]}
Clinical data saved to ../../output/preprocess/Melanoma/

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 49372
First 20 gene/probe identifiers:
Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be Affymetrix probe IDs (format like "11715100_at")
# rather than standard human gene symbols (like BRCA1, TP53, etc.)
# These probe IDs need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation that correspond to probe IDs and gene symbols
probe_id_col = 'ID'
gene_symbol_col = 'Gene Symbol'

# 2. Extract the mapping dataframe containing probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)

# 3. Load the gene expression data
genetic_data = get_genetic_data(matrix_file)

# 4. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(genetic_data, gene_mapping)

# Print information about the resulting gene data
print(f"Converted from {len(genetic_data)} probes to {len(gene_data)} genes")
print("First 10 gene symbols:")
print(gene_data.index[:10])


Converted from 49372 probes to 19963 genes
First 10 gene symbols:
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS',
       'AACS', 'AACSP1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Extract clinical features using the clinical data from step 1
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Extract clinical features using the convert_trait function from step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # From step 2
    convert_trait=convert_trait,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 5. Determine if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from juvenile myositis (JM) and childhood-onset lupus (cSLE) skin biopsies."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")

Top 10 gene indices before normalization: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AACSP1']
Top 10 gene indices after normalization: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AACSP1']
Shape of normalized gene data: (19758, 83)


Saved normalized gene data to ../../output/preprocess/Melanoma/gene_data/GSE148319.csv


Saved clinical data to ../../output/preprocess/Melanoma/clinical_data/GSE148319.csv
Shape of linked data: (83, 19759)


Shape of linked data after handling missing values: (83, 19759)
Quartiles for 'Melanoma':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Melanoma' in this dataset is severely biased.

Dataset validation failed. Final linked data not saved.
